Downloading xBD from the Google Bucket
Link instance to bucket (https://cloud.google.com/compute/docs/disks/gcs-buckets#api)
gcloud init # (exec in terminal; click 1 y 1)
pip install --upgrade google-cloud-storage
create account service credentials ( https://cloud.google.com/docs/authentication/production#cloud-console ) and copy them into the notebook

In [ ]:
import numpy as np
import sys
import os
import numpy as np
from PIL import Image
from os.path import isfile, join
import csv

# Link account service credentils to this notebook
def explicit():
    from google.cloud import storage

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'FDL US 2020 EIE-7be61e397b4f.json')

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)
explicit()

from google.cloud import storage

storage_client = storage.Client() 
bucket = storage_client.get_bucket("eie_xview_raw") # Useful: Get names of buckets with storage_client.list_buckets()
print(bucket)

def download_all(prefix, dl_dir):
    try:
        original_umask = os.umask(0)
        if not os.path.exists(dl_dir):
            os.makedirs(dl_dir, mode=0o777)
    finally:
        os.umask(original_umask)
#     allpaths = bucket.list_blobs(prefix=prefix)
    blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
    
#     f = open(dl_dir +'paths.csv', 'w')
#     with f:
#         writer = csv.writer(f)
#         for row in [allpaths]:
#             row = list(row)[21:-19]
#             writer.writerow(row) #We save a .csv to link with google AI labeling tool to create segmentation masks later.
    
    #loops trough all files in the bucket under the prefix and downloads them into the dl_dir
    for blob in blobs:
        filename = blob.name.replace('/', '_')
        blob.download_to_filename(dl_dir + filename)
        
for phase in ['train/', 'test/', 'val/']:        
    try:
        original_umask = os.umask(0)
        dl_dir = './xBD/bigtiles/'+phase
        if not os.path.exists(dl_dir):
            os.makedirs(dl_dir, mode=0o777)
    finally:
        os.umask(original_umask)
#Trainset
print("Downloading train set")
prefix = 'training-set/train/images/hurricane'
dl_dir = './xBD/bigtiles/train/'
download_all(prefix, dl_dir)

    
#Testset
print("Downloading test set")
prefix = 'test-set/test_images_labels_targets/test/images/hurricane'
dl_dir = './xBD/bigtiles/test/'
download_all(prefix, dl_dir)

#validation
print("Downloading validation set")
prefix = 'holdout-set/hold_images_labels_targets/hold/images/hurricane'
dl_dir = './xBD/bigtiles/val/'
download_all(prefix, dl_dir)
print('Done')


[<Bucket: eie_floods>, <Bucket: eie_internal_bigquery_repository>, <Bucket: eie_xview_processed>, <Bucket: eie_xview_raw>]
<Bucket: eie_xview_raw>


In [ ]:
#Move post disaster into 'train_B' and pre disaster into 'train_A' (data file system used by Pix2pixHD).
!cd xBD/bigtiles/ ; mkdir train_A train_B train_AB test_A test_B test_AB val_A val_B val_AB ; mv train/*post* train_B/ ; mv train/*pre* train_A/ ; mv test/*post* test_B/ ; mv test/*pre* test_A/ ; mv val/*post* val_B/ ; mv val/*pre* val_A/

## Create aligned (concatenated) side-by-side .png files for the trainset of bicycleGAN

In [ ]:
import numpy as np
import sys
import os
import numpy as np
from PIL import Image
from os.path import isfile, join
import csv

def A_and_B_to_AB(path="./xBD/bigtiles/"):
    print('Make sure files are named in a way that sorting them alphabetically \
        both in *_A and *_B folders produces the same order (i.e., ideally, \
        files are named equally but placed in different *_A and *_B folders')
        
    for folder in ['train','test','val']:
        A_dir = path + folder + '_A/'
        B_dir = path + folder + '_B/'
        AB_dir = path + folder + '_AB/'
        print('Generating ',AB_dir)
        try:
            os.mkdir(AB_dir)
        except FileExistsError:
            pass
        train_A_list = [f.strip() for f in os.listdir(A_dir) if isfile(join(A_dir, f)) and valid_extension(join(A_dir, f))]
        train_A_list.sort()

        train_B_list = [f.strip() for f in os.listdir(B_dir) if isfile(join(B_dir, f)) and valid_extension(join(B_dir, f))]
        train_B_list.sort()

        for img_path_A, img_path_B in zip(train_A_list, train_B_list):
            # assert img_path_A == img_path_B, 'Make sure files are named in a \
            # way that sorting them alphabetically both in *_A and *_B folders \
            # produces the same order (i.e., ideally, files are named equally but \
            # placed in different *_A and *_B folders: {}\n{}'.format(img_path_A, img_path_B)
            print('Generating {} image AB for {}'.format(folder,img_path_A))
            images = [Image.open(x) for x in [A_dir + img_path_A, B_dir+img_path_B]]
            widths, heights = zip(*(i.size for i in images))

            total_width = sum(widths)
            max_height = max(heights)

            new_im = Image.new('RGB', (total_width, max_height))

            x_offset = 0
            for im in images:
                new_im.paste(im, (x_offset,0))
                x_offset += im.size[0]

            new_im.save(path+folder+'_AB/'+img_path_A)
        print('Done with ' +folder)
    
        
A_and_B_to_AB("./xBD/bigtiles/")

## Create 256x256 dataset

In [ ]:
def bigtiles_to_smalltiles(bigtiles="./xBD/bigtiles/", small_path="./xBD/smalltiles/",output_size=256):
    folders = ['train_A', 'train_B', 'train_AB', 'test_A', 'test_B', 'test_AB', 'val_A', 'val_B', 'val_AB']
    for folder in folders:
        directory = small_path+folder
        if not os.path.exists(directory):
            os.makedirs(directory, mode=0o777)
        try:
            original_umask = os.umask(0)
        finally:
            os.umask(original_umask)
        
    def crop_tiles(image):
        image = np.array(image)
        width = image.shape[0]
        N = width/output_size
        h_tiles = np.split(image, N)
        
        images = []
        for tile in h_tiles:
            images.append(np.split(tile, N, axis=1))
        images = [item for sublist in images for item in sublist] #flatten nested list
        return images
    
    def load_crop_save(list_files, path):
        for i in range(len(list_files)):
            image = Image.open("./xBD/bigtiles/"+path+list_files[i])
            images = crop_tiles(image)
            for num , im in enumerate(images): # ToDo, not tested but should be not num but str(im) as otherwise later we require in *_AB
                # toDo: rm 0*.png;rm 1*.png; rm 2*.png;rm  3*.png; rm 4*.png;rm 5*.png;rm 6*.png; rm 7*.png ; rm 8*.png; rm 9*.png; 
                name = list_files[i][:-4]+str(im).zfill(2)+'.png'
                new_im = Image.fromarray(im)
                new_im.save('./xBD/smalltiles/'+path+name)
    
    for dataset in ['train','test','val']:
        A_dir = bigtiles + dataset + '_A/'
        B_dir = bigtiles + dataset + '_B/'

        train_A_list = [f for f in os.listdir(A_dir) if isfile(join(A_dir, f))]
        train_A_list.sort()
        print("Cropping "+ dataset+ " A")
        load_crop_save(train_A_list, dataset+'_A/')

        train_B_list = [f for f in os.listdir(B_dir) if isfile(join(B_dir, f))]
        train_B_list.sort()
        print("Cropping "+dataset+" B")
        load_crop_save(train_B_list, dataset+'_B/')
    print('Done with '+dataset)


def extension(filename):
    return filename.split('.')[-1]

def valid_extension(filename):
    return extension(filename) in set(['jpg','png','jpeg'])

def A_and_B_to_AB(path="./xBD/bigtiles/"):
    print('Make sure files are named in a way that sorting them alphabetically \
        both in *_A and *_B folders produces the same order (i.e., ideally, \
        files are named equally but placed in different *_A and *_B folders')
        
    for folder in ['train','test','val']:
        A_dir = path + folder + '_A/'
        B_dir = path + folder + '_B/'
        AB_dir = path + folder + '_AB/'
        print('Generating ',AB_dir)
        try:
            os.mkdir(AB_dir)
        except FileExistsError:
            pass
        train_A_list = [f.strip() for f in os.listdir(A_dir) if isfile(join(A_dir, f)) and valid_extension(join(A_dir, f))]
        train_A_list.sort()

        train_B_list = [f.strip() for f in os.listdir(B_dir) if isfile(join(B_dir, f)) and valid_extension(join(B_dir, f))]
        train_B_list.sort()

        for img_path_A, img_path_B in zip(train_A_list, train_B_list):
            # assert img_path_A == img_path_B, 'Make sure files are named in a \
            # way that sorting them alphabetically both in *_A and *_B folders \
            # produces the same order (i.e., ideally, files are named equally but \
            # placed in different *_A and *_B folders: {}\n{}'.format(img_path_A, img_path_B)
            print('Generating {} image AB for {}'.format(folder,img_path_A))
            images = [Image.open(x) for x in [A_dir + img_path_A, B_dir+img_path_B]]
            widths, heights = zip(*(i.size for i in images))

            total_width = sum(widths)
            max_height = max(heights)

            new_im = Image.new('RGB', (total_width, max_height))

            x_offset = 0
            for im in images:
                new_im.paste(im, (x_offset,0))
                x_offset += im.size[0]

            new_im.save(path+folder+'_AB/'+img_path_A)
        print('Done with ' +folder)
    
bigtiles_to_smalltiles()

A_and_B_to_AB("./xBD/bigtiles/")
    